In [4]:
!pip install simpletransformers

     |████████████████████████████████| 163kB 8.1MB/s 
     |████████████████████████████████| 204kB 16.4MB/s 
     |████████████████████████████████| 573kB 19.6MB/s 
     |████████████████████████████████| 3.8MB 15.5MB/s 
     |████████████████████████████████| 1.0MB 58.9MB/s 
     |████████████████████████████████| 890kB 55.2MB/s 
  Created wheel for seqeval: filename=seqeval-0.0.12-cp36-none-any.whl size=7424 sha256=ef5a6541168a05a773dcab18b0b9fd624b37dbc2862653d9fd88ebf3c368cdac
  Stored in directory: /root/.cache/pip/wheels/4f/32/0a/df3b340a82583566975377d65e724895b3fad101a3fb729f68
  Created wheel for sacremoses: filename=sacremoses-0.0.41-cp36-none-any.whl size=893334 sha256=0eb38668352ea33264a761455f63758c0c75e75b75a92e688412d407f84b86b9
  Stored in directory: /root/.cache/pip/wheels/22/5a/d4/b020a81249de7dc63758a34222feaa668dbe8ebfe9170cc9b1
Successfully built seqeval sacremoses
ERROR: transformers 2.8.0 has requirement tokenizers==0.5.2, but you'll have tokenizers 0.7.0 which

In [0]:
TRAIN_PATH = 'Train.csv'
TEST_PATH = 'Test.csv'
SAMPLE_SUB_PATH = 'SampleSubmission.csv'

In [0]:
import numpy as np
import pandas as pd
from sklearn.metrics import *
from sklearn.model_selection import *
#Miscellaneous
from tqdm import tqdm
import warnings
warnings.simplefilter('ignore')
import gc
from scipy.special import softmax

from simpletransformers.classification.classification_model import ClassificationModel

In [0]:
from sklearn.metrics import mean_squared_error as mse

In [0]:
train = pd.read_csv(TRAIN_PATH)
test = pd.read_csv(TEST_PATH)
sample_sub = pd.read_csv(SAMPLE_SUB_PATH)
ID_COL, TARGET_COL = sample_sub.columns.tolist()

In [30]:
train.head()

,tweet_id,safe_text,label,agreement
0,CL1KWCMY,Me &amp; The Big Homie meanboy3000 #MEANBOY #M...,0.0,1.0
1,E3303EME,I'm 100% thinking of devoting my career to pro...,1.0,1.0
2,M4IVFSMS,"#whatcausesautism VACCINES, DO NOT VACCINATE Y...",-1.0,1.0
3,1DR6ROZ4,I mean if they immunize my kid with something ...,-1.0,1.0
4,J77ENIIE,Thanks to <user> Catch me performing at La Nui...,0.0,1.0


In [31]:

train['safe_text'].apply(lambda x: len(x)).describe()

count    10001.000000
mean        99.902810
std         29.893888
min          1.000000
25%         79.000000
50%        107.000000
75%        122.000000
max        153.000000
Name: safe_text, dtype: float64

In [32]:
train['label'].value_counts()

 0.000000    4908
 1.000000    4053
-1.000000    1038
 0.666667       1
Name: label, dtype: int64

In [0]:
train['label'][~train['label'].isin([0, -1, 1])] = -1

In [34]:
train.isnull().sum()

tweet_id     0
safe_text    0
label        0
agreement    2
dtype: int64

In [0]:
train = train.dropna()

In [36]:
train.isnull().sum()

tweet_id     0
safe_text    0
label        0
agreement    0
dtype: int64

In [37]:
test.head()

,tweet_id,safe_text
0,00BHHHP1,<user> <user> ... &amp; 4 a vaccine given 2 he...
1,00UNMD0E,Students starting school without whooping coug...
2,01AXPTJF,"I'm kinda over every ep of <user> being ""rippe..."
3,01HOEQJW,How many innocent children die for lack of vac...
4,01JUKMAO,"CDC eyeing bird flu vaccine for humans, though..."


In [38]:
train[train['label'] == -1]

,tweet_id,safe_text,label,agreement
2,M4IVFSMS,"#whatcausesautism VACCINES, DO NOT VACCINATE Y...",-1.0,1.000000
3,1DR6ROZ4,I mean if they immunize my kid with something ...,-1.0,1.000000
24,89AB846O,<user> #CDC lied and hid data that black boys ...,-1.0,1.000000
25,XSXFN1C8,<user> vaccines causing autism,-1.0,1.000000
35,6PMH7C56,"<user> <user> Other than that, his defense is ...",-1.0,0.333333
...,...,...,...,...
9913,IOQF5U8D,"I'm not saying to not give vaccines, I am just...",-1.0,0.333333
9917,AL40FCZO,<user> please stop pushing this bs flu vaccine...,-1.0,1.000000
9946,LO6KLY0L,kendyl wont be allowed in school monday if i d...,-1.0,0.666667
9947,0WKSH5HI,Gardasil Vaccine Victims and Deaths - Did You ...,-1.0,0.666667


In [39]:
test.isnull().sum()

tweet_id     0
safe_text    1
dtype: int64

In [0]:
test['safe_text'] = test['safe_text'].fillna('xxxxxx')

In [41]:
test.isnull().sum()

tweet_id     0
safe_text    0
dtype: int64

**Lets see some of the tweets of each kind**

**1. Positive**

In [42]:
train[train['label'] == 1]['safe_text'].values[:5]

array(["I'm 100% thinking of devoting my career to proving autism isn't caused by vaccines due to the IDIOTIC posts I've seen about World Autism Day",
       '<user> a nearly 67 year old study when mental health studies and vaccines were relatively in their infancies that has been refuted?',
       'Study of more than 95,000 kids finds no link between MMR vaccine and autism <url>',
       'psa: VACCINATE YOUR FUCKING KIDS',
       'Coughing extra on the shuttle and everyone thinks I have the measles. 😂 #VaccinateYourKids'],
      dtype=object)

**2. Neutral**

In [43]:
train[train['label'] == 0]['safe_text'].values[:5]

array(['Me &amp; The Big Homie meanboy3000 #MEANBOY #MB #MBS #MMR #STEGMANLIFE @ Stegman St. <url>',
       'Thanks to <user> Catch me performing at La Nuit NYC 1134 1st ave. Show starts at 6! #jennifair #mmr… <url>',
       "<user> @ this point I have 2 text, butw/Bon Jovi cover playin @ Alibi's hope U can come out 2 MMR BBQ<user> will b there!",
       'My prediction, vaccine exemption in Arizona will end soon. To much money is being lost by big pharma.',
       '1$Mug Noche <user> #mmr #mixmasterrod #dcdj #mmr   @ Mad Hatter <url>'],
      dtype=object)

**3. Negative**

In [44]:
train[train['label'] == -1]['safe_text'].values[:5]

array(['#whatcausesautism VACCINES, DO NOT VACCINATE YOUR CHILD',
       "I mean if they immunize my kid with something that won't secretly kill him years down the line then I'm all for it, but I don't trust that",
       '<user> #CDC lied and hid data that black boys have an 340% uncreased risk of developing autism after MMR #CDCwhistleblower help!?!?',
       '<user> vaccines causing autism',
       '<user> <user> Other than that, his defense is not against vaccines being harmful - but the American life being unhealthy.'],
      dtype=object)

Lets go to the modelling part. **Simple Transformers** is extremely simple to use, and switching architectures requires only the change in argument name.

In [0]:
def get_model(model_type, model_name, n_epochs = 2, train_batch_size = 112, eval_batch_size = 144, seq_len = 134, lr = 1.8e-5):
  model = ClassificationModel(model_type, model_name,num_labels=1, args={'train_batch_size':train_batch_size,"eval_batch_size": eval_batch_size,
                                                                         'reprocess_input_data': True,
                                                                    'overwrite_output_dir': True, 'fp16': False,
                                                                    'do_lower_case': False, 'num_train_epochs': n_epochs,
                                                                    'max_seq_length': seq_len,'regression': True,
                                                                    'manual_seed': 2, "learning_rate":lr,
                                                                    "save_eval_checkpoints": False,
                                                                    "save_model_every_epoch": False,})
  return model 

In [0]:
tmp = pd.DataFrame()
tmp['text'] = train['safe_text']
tmp['labels'] = train['label']

In [0]:
tmp_test = test[['safe_text']].rename({'safe_text': 'text'}, axis=1)
tmp_test['labels'] = 0

In [48]:
tmp['labels'].value_counts()

 0.0    4908
 1.0    4053
-1.0    1038
Name: labels, dtype: int64

In [0]:
tmp_trn, tmp_val = train_test_split(tmp, test_size=0.15, random_state=2)

In [50]:
model = get_model('roberta', 'roberta-base', n_epochs=3)
model.train_model(tmp_trn)
preds_val = model.eval_model(tmp_val)[1]
preds_val = np.clip(preds_val, -1, 1)
print(f"RMSE: {mse(tmp_val['labels'], preds_val)**0.5}")
test_preds = model.eval_model(tmp_test)[1]
test_preds = np.clip(test_preds, -1, 1)
pv_1 = preds_val
pt_1 = test_preds

Running loss: 0.309141


Running loss: 0.222377


Running loss: 0.263232




RMSE: 0.5252905752629805


In [51]:
model = get_model('roberta', 'roberta-large', n_epochs=1, train_batch_size=16, eval_batch_size=16)
model.train_model(tmp_trn)
preds_val = model.eval_model(tmp_val)[1]
preds_val = np.clip(preds_val, -1, 1)
print(f"RMSE: {mse(tmp_val['labels'], preds_val)**0.5}")
test_preds = model.eval_model(tmp_test)[1]
test_preds = np.clip(test_preds, -1, 1)
pv_2 = preds_val
pt_2 = test_preds

Running loss: 0.019080




RMSE: 0.510520832024609


In [52]:
model = get_model('roberta', 'roberta-large', n_epochs=2, train_batch_size=16, eval_batch_size=16, lr = 1.5e-5)
model.train_model(tmp_trn)
preds_val = model.eval_model(tmp_val)[1]
preds_val = np.clip(preds_val, -1, 1)
print(f"RMSE: {mse(tmp_val['labels'], preds_val)**0.5}")
test_preds = model.eval_model(tmp_test)[1]
test_preds = np.clip(test_preds, -1, 1)
pv_3 = preds_val
pt_3 = test_preds

Running loss: 0.206463


Running loss: 0.041137




RMSE: 0.47697794653788994


In [53]:
model = get_model('roberta', 'roberta-large', n_epochs=3, train_batch_size=16, eval_batch_size=16, lr = 5e-6)
model.train_model(tmp_trn)
preds_val = model.eval_model(tmp_val)[1]
preds_val = np.clip(preds_val, -1, 1)
print(f"RMSE: {mse(tmp_val['labels'], preds_val)**0.5}")
test_preds = model.eval_model(tmp_test)[1]
test_preds = np.clip(test_preds, -1, 1)
pv_4 = preds_val
pt_4 = test_preds

Running loss: 0.390808


Running loss: 0.056846


Running loss: 0.936049




RMSE: 0.48014712584116526


In [86]:
pv = ((pv_1 * 0.3 + pv_2 * 0.7) * 0.35 + pv_3*0.65)*0.7 + pv_4*0.3
print(f"RMSE: {mse(tmp_val['labels'], pv)**0.5}")

RMSE: 0.46229822081934346


In [0]:
tp = ((pt_1 * 0.3 + pt_2 * 0.7) * 0.35 + pt_3*0.65)*0.7 + pt_4*0.3

In [88]:
pd.Series(tp).describe()

count    5177.000000
mean        0.358857
std         0.486373
min        -0.869760
25%         0.017804
50%         0.254047
75%         0.882496
max         1.000000
dtype: float64

In [0]:
final_test_preds = tp
preds_df_final = test[[ID_COL]]
preds_df_final[TARGET_COL] = final_test_preds
SUB_FILE_NAME = 'roberta_ensemble.csv'
preds_df_final.to_csv(SUB_FILE_NAME, index=False)

In [90]:
preds_df_final.head()

,tweet_id,label
0,00BHHHP1,-0.561573
1,00UNMD0E,0.234386
2,01AXPTJF,0.065135
3,01HOEQJW,0.948843
4,01JUKMAO,0.285690


In [91]:
preds_df_final.tail()

,tweet_id,label
5172,ZXVVNC5O,0.967264
5173,ZYIANVI8,0.075747
5174,ZYITEHAH,0.533786
5175,ZZ3BMBTG,1.000000
5176,ZZIYCVNH,-0.301241


In [92]:
preds_df_final.shape

(5177, 2)

In [93]:
preds_df_final['label'].describe()

count    5177.000000
mean        0.358857
std         0.486373
min        -0.869760
25%         0.017804
50%         0.254047
75%         0.882496
max         1.000000
Name: label, dtype: float64

In [94]:
preds_df_final.isnull().sum()

tweet_id    0
label       0
dtype: int64

In [0]:
from google.colab import files
files.download(SUB_FILE_NAME)